# 1. 필요한 패키지

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import splitfolders

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
def json_path(json_path):
    """
    csv 파일을 DataFrame으로 불러 읽어오는 함수
    """
    with open(json_path, 'r') as file:
        data = json.load(file)
    return data

# 1-1. LSTM AutoEncoder

- LSTM AutoEncoder를 이용한 시계열 기반 데이터 이상탐지 데이터전처리 단계

In [ ]:
def split_train_test(df):
    """
    data를 train_data와 test_data로 split하는 함수로, 전체 data의 70%를 train_data로 처리한다.
    train값에는 정상인 데이터만 필요하기 때문에 정상인 데이터만 뽑아주는 과정을 거친다.
    
    input에 dataframe이 들어가며, output으로는 train_data, test_data가 나오게 된다.
    """
    df.drop(columns = ["Time"], axis = 1, inplace = True)
    train_data = df[:round(len(df)*0.7)]
    train_data = train_data[train_data['label'] == 0]
    test_data = df[round(len(df)*0.7):]
    return train_data, test_data

In [ ]:
def LSTM_train_data(train_data):
    """
    X_train은 train_data에서 label을 제외한 dataframe이다.
    X_train을 표준화한다.
    y_train은 train_data에서 label 값만 불러와 리스트화한다.
    
    input에 split_train_test에서 만든 train_data가 들어가며, output으로는 X_train, y_train, X_train_column이 나오게 된다.
    """
    X_train = train_data.drop([train_data.columns[-1]], axis = 1)
    X_train_column = X_train.columns
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    y_train = list(train_data['label'])
    return X_train, y_train, X_train_column

In [ ]:
def LSTM_test_data(test_data):
    """
    X_test는 test_data에서 label을 제외한 dataframe이다.
    X_test를 표준화한다.
    y_test는 test_data에서 label 값만 불러와 리스트화한다.
    
    input에 split_train_test에서 만든 test_data가 들어가며, output으로는 X_test, y_test가 나오게 된다.
    """
    X_test = test_data.drop([test_data.columns[-1]], axis = 1)
    scaler = StandardScaler()
    X_test = scaler.fit_transform(X_test)
    y_test = list(test_data['label'])
    return X_test, y_test

In [ ]:
def reshape(X_train, X_test):
    """
    모델에 맞게 데이터 형태 바꿔준다.
    
    input에 X_train, X_test data가 들어가며, output으로는 X_train, X_test가 나오게 된다.
    """
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    return X_train, X_test

# 1.2 IForest

- IForest를 이용한 시계열 기반 데이터 이상탐지 데이터전처리 단계

In [ ]:
def pick_column(df):
    """
    각각 필요한 data만 뽑는다.
    
    input에 dataframe이 들어가며, output으로는 X_data, y_data가 나오게 된다.
    """
    
    y_data = df['label']
    X_data = df.drop(['Time', 'label'], axis=1)

    return X_data, y_data